In [1]:
import re
from random import uniform
from collections import defaultdict

r_alphabet = re.compile(u'[а-яА-Я0-9-]+|[.,:;?!]+')

def gen_lines(corpus):
    data = open(corpus)
    for line in data:
        yield line.lower()

def gen_tokens(lines):
    for line in lines:
        for token in r_alphabet.findall(line):
            yield token

def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?':
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$'
        else:
            t0, t1 = t1, t2

def train(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)

    bi, tri = defaultdict(lambda: 0.0), defaultdict(lambda: 0.0)

    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1

    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]
    return model

def generate_sentence(model):
    phrase = ''
    t0, t1 = '$', '$'
    while 1:
        t0, t1 = t1, unirand(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
    return phrase.capitalize()

def unirand(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return token


-- а николай мне ... я принимаю ваше предложение ... она не входила ко мне?
-- соня, крестясь.
Увидав мундир училища, которое следует обыкновенно за оконченным серьезным и настоящим миром.
Чаще и чаще, и через него, и князь андрей, умывшись и одевшись, дома.
5 немедленно средства употреблены будут для восстановления границ народов был так велик ростом, очень бы хотелось.
Это был первый раз, раз сказанные пьером.
Княгиня марья васильевна мужу.
И любила.
- сказал старик и, обращаясь к подошедшему офицеру.
--, 5 -- и князь андрей убит, а всего более за мою руку держись, уронишь так, чтобы у вас, оставить или не доедете до армии и о последнем событии на смотру, энглизированной кобыле и, как он был спокоен, он вернулся, и аббат морио и многие не могла спать и несколько людей присоединились к двигавшейся толпе.


In [3]:
model = train('tolstoy.txt')
for i in range(10):
    print (generate_sentence(model))

Ростов почувствовал, что кричавший братцы!
-- государь тут.
- зашей и эти нити, но, заметив, что, кроме анатоля, того, что мы сделаем, что весь интерес жизни.
Явилось, как бы мне это письмо, положил морду мне на постели, казалось, что он так дорожил прежде.
-- что вы не знакомы с?
Мельник уже два года в деревне.
-- помнишь, как у детей в том, что она увидит гроб, то она не отвечала и пошла в девичью; в сравнении с этой точки зрения, с головой и с своим безумием самообожания, с трудом опустился на веревках фабричный, неестественно опустив голову, когда он говорил это на деле.
Нет, нет причины, и никто не любит и уважает.
Княжна марья не говорила иначе, чем прежде, остались за столом князь чаще всего обращался к нему, испугала ростова.
С каратаевым и лиловой кривоногой собакой, которая более всего из рассказа князя андрея, несмотря на то, что для того чтобы атаковать, недостаточно желания сделать это усилие и лег, как чуть-чуть, легонько дотронуться до груди.


In [5]:
import pandas as pd

In [6]:
df1 = pd.read_csv('24.csv')
df2 =pd.read_csv('lenta.csv')
df3 = pd.read_csv('vesti.csv')
df = pd.concat([df1,df2,df3])
del df1
del df2
del df3
df = df.dropna()

In [19]:
text = ' \n '.join(df.text)

In [22]:
file1 = open("MyFile.txt","w") 
file1.write(text)
file1.close() 

In [23]:
model = train('MyFile.txt')
for i in range(10):
    print (generate_sentence(model))

Следующее евровидение пройдет с 14 по словам специалистов, прибор регистрации параметров полета подвергся интенсивному температурному воздействию и получил свою демократию, потому что после избрания главой правительства чечни в 2015 году украина и 1, забил 32 мяча и отдал ему свое сердце он завещал похоронить в польше, германии, раскритиковав политику украинизации страны.
Мама ирина всегда полностью соблюдала все рекомендации по порядку возбуждения дел, возбужденных по статьям хулиганство и побои.
К 20 дням ареста за вторжение в германию в 1882 году он перенес несколько сердечных приступов.
Преподавателя по музыке задержали правоохранители.
В настоящее время под наблюдением врачей находятся 62 человека, выяснить, чем дольше ребенок находится в деликатной ситуации: есть опасения нехватки сырья, а от того, что изучает размер ущерба, а фронтальный 1920х1080 точек 30 60, а также оказать помощь при госпитализации подъяпольского.
Семь процентов россиян.
Для детей старше 16 лет.
.
Пострадавши